# Tests

In [5]:
using LinearAlgebra
using Turing
using LimberJack
using PythonCall
using NPZ;

In [6]:
np = pyimport("numpy");

In [7]:
meta = np.load("../data/FD/K1K_DELS_DESY1_eBOSS_CMBk_meta.npz")
tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
pairs_ids = pyconvert(Vector{Vector{Int}}, meta["pairs_ids"])
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"])
inv_cov_tot = pyconvert(Matrix{Float64}, meta["inv_cov"]);

In [8]:
files = npzread("../data/FD/K1K_DELS_DESY1_eBOSS_CMBk_files.npz");

In [9]:
pars = np.loadtxt("../chains/carlos_chains/cl_cross_corr_v3_DES_K1000_all_mag_correctMag/cl_cross_corr_v3_DES_K1000_all_mag_correctMag.bestfit")
pars = pyconvert(Vector{Float64}, pars);

In [10]:
bestfit = np.load("../chains/carlos_chains/cl_cross_corr_v3_DES_K1000_all_mag_correctMag/cl_cross_corr_bestfit_info.npz")
length(bestfit["cls"])

665

In [12]:
Ωm = pars[1] + pars[2]
s8 = pars[46]
function make_cls(pars; Ωm=Ωm, s8=s8)
    nuisances = Dict("DESgc__0_0_b" => pars[6],
                     "DESgc__1_0_b" => pars[7],
                     "DESgc__2_0_b" => pars[8],
                     "DESgc__3_0_b" => pars[9],
                     "DESgc__4_0_b" => pars[10],
                     "DESgc__0_0_dz" => pars[11],
                     "DESgc__1_0_dz" => pars[12],
                     "DESgc__2_0_dz" => pars[13],
                     "DESgc__3_0_dz" => pars[14],
                     "DESgc__4_0_dz" => pars[15],
        
                     "A_IA" => pars[16],
                     "alpha_IA" => pars[17],
        
                     "DESgc__0_e_m" => pars[18],
                     "DESgc__1_e_m" => pars[19],
                     "DESgc__2_e_m" => pars[20],
                     "DESgc__3_e_m" => pars[21],
                     "DESgc__0_e_dz" => pars[22],
                     "DESgc__1_e_dz" => pars[23],
                     "DESgc__2_e_dz" => pars[24],
                     "DESgc__3_e_dz" => pars[25],
        
                     "eBOSS__0_0_b" => pars[26],
                     "eBOSS__1_0_b" => pars[27],

                     "DECALS__0_0_b" => pars[28],
                     "DECALS__1_0_b" => pars[29],
                     "DECALS__2_0_b" => pars[30],
                     "DECALS__3_0_b" => pars[31],
                     "DECALS__0_0_dz" => pars[32],
                     "DECALS__1_0_dz" => pars[33],
                     "DECALS__2_0_dz" => pars[34],
                     "DECALS__3_0_dz" => pars[35],
                    
                     "KiDS1000__0_e_m" => pars[36],
                     "KiDS1000__1_e_m" => pars[37],
                     "KiDS1000__2_e_m" => pars[38],
                     "KiDS1000__3_e_m" => pars[39],
                     "KiDS1000__0_e_dz" => pars[40],
                     "KiDS1000__1_e_dz" => pars[41],
                     "KiDS1000__2_e_dz" => pars[42],
                     "KiDS1000__3_e_dz" => pars[43])

    Ωb = pars[1]
    h = pars[5]
    ns = pars[4]
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="emulator", Pk_mode="Halofit")

    return Theory(cosmology,
                  tracers_names, pairs,
                  pairs_ids, idx, files;
                  Nuisances=nuisances)

end

make_cls (generic function with 1 method)

## Xi2

In [13]:
best_pred = make_cls(pars);

In [14]:
best_Xi = dot((best_pred .- data_vector), inv_cov_tot  * (best_pred .- data_vector))

1403.5052885973644

In [15]:
reduced_best_Xi = round(best_Xi/(length(data_vector)-43), digits=3)

1.139

## Prior sampling

In [16]:
N_samples = 100
cls_preds = zeros(N_samples, length(data_vector))
for i in 1:N_samples
    Ωm = rand(Uniform(0.1, 0.6))
    s8 = rand(Uniform(0.6, 0.9))
    cls_preds[i,:] = make_cls(pars; Ωm=Ωm, s8=s8)
end

In [20]:
cls_preds_m, cls_preds_s = mean(cls_preds, dims=1), std(cls_preds, dims=1);

In [22]:
npzwrite("jaime_cls.npz", Dict("cls_m" => cls_preds_m,
                               "cls_s" => cls_preds_s))

## Turing

In [10]:
@model function model(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 = 0.81 #~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)

    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, s8, s8,
                                     tk_mode="emulator",
                                     Pk_mode="Halofit")
    theory = Theory(cosmology,
                    tracers_names, pairs,
                    pairs_ids, idx, files)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

In [11]:
iterations = 300
nadapts = 100;

In [12]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:24


LoadError: InterruptException:

In [10]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/51xgc/src/hamiltonian.jl:47
Sampling:   1%|▌                                        |  ETA: 0:08:52┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/51xgc/src/hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:01:01


LoadError: InterruptException:

In [4]:
files = npzread("../data/FD/K1K_DELS_DESY1_eBOSS_CMBk_files.npz")
products = npzread("products.npz")

Dict{String, Array} with 5 entries:
  "idx"         => [0, 5, 13, 23, 34, 47, 52, 57, 62, 67  …  1077, 1099, 1121, …
  "cls_m"       => [0.000229969 7.55142e-5 … 3.71104e-10 3.14893e-10]
  "err"         => [5.919e-5, 1.57008e-5, 6.64192e-6, 3.82552e-6, 2.56294e-6, 2…
  "cls_s"       => [0.000216359 3.16836e-5 … 2.44088e-10 2.0759e-10]
  "data_vector" => [0.000190242, 7.39128e-5, 4.26125e-5, 2.22966e-5, 2.17688e-5…

In [3]:
figure, axis = plt.subplots(5, figsize=(5,14))
l = 1
for i in 0:4
    axis[i+1].errorbar(files[string("ls_1", i, "1", i)], 
                     data[idx[l]:idx[l+1]-1], 
                     yerr=err[idx[l]:idx[l+1]-1], fmt="gs", label="Data")
    axis[i+1].plot(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1],
                   "bo-", label="Preds")
    axis[i+1].fill_between(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1].+cls_preds_s[idx[l]:idx[l+1]-1],
                   cls_preds_m[idx[l]:idx[l+1]-1].-cls_preds_s[idx[l]:idx[l+1]-1],
                   color="b", alpha=.5)
    axis[i+1].set_title(string("GC", i, "GC", i))
    l += 1
end
plt.legend()
plt.show()
print(l)

LoadError: UndefVarError: files not defined